In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import ase.io
from ipywidgets import interact, interactive, IntSlider, Layout

In [ ]:
### Input folder
folder = "examples/Q-0.12K0.15/"
afm = "Amp1.40/"
geom = "examples/Q-0.12K0.15/geom.xyz"

In [ ]:
def read_PPPos(filename):
    """!
    @brief Loads the positions obtained from the probe particle model.

    In this case 'filename' refers to the head of files whose tail is assumed
    to be of the form '*_x.npy', '*_y.npy' and '*_z.npy.'
    
    The units are in Angstrom.

    @param filename Name of the start of the file without tail.

    @return List of positions in shape of mgrid.
            Matrix containing information on the unrelaxed grid.
    """
    disposX = np.transpose(np.load(filename+"_x.npy")).copy()
    disposY = np.transpose(np.load(filename+"_y.npy")).copy()
    disposZ = np.transpose(np.load(filename+"_z.npy")).copy()
    # Information on the grid
    lvec = (np.load(filename+"_vec.npy"))
    # Stack arrays to form 4-dimensional array of size [3, noX, noY, noZ]
    dispos = (disposX,disposY,disposZ)

    return dispos, lvec

In [ ]:
# Read relaxed position for C and O
pos, lVec = read_PPPos(folder+"PPpos")
disp, lVec = read_PPPos(folder+"PPdisp")
# AFM image
df = np.load(folder+afm+"df.npy")
# Dimension and plotting information
gridDim = np.shape(df)[::-1]
extent = [lVec[0,0], lVec[1,0], lVec[0,1], lVec[2,1]]
x_ref = np.linspace(lVec[0,0],lVec[1,0],gridDim[0])
y_ref = np.linspace(lVec[0,1],lVec[2,1],gridDim[1])
# Atoms
colorDic = {'H': 'gray', 'C': 'black', 'N': 'blue', 'Au': 'gold'}
atoms = ase.io.read(geom)
colors = [colorDic[atom] for atom in atoms.get_chemical_symbols()]
# Plot limits
disx_lim = [lVec[0,0],lVec[1,0]]
disy_lim = [lVec[0,1],lVec[2,1]]

In [ ]:
x_slider = IntSlider(description="$x$ Position", value=gridDim[0]//2,min=0,max=gridDim[0]-1, layout=Layout(width='60%'))
y_slider = IntSlider(description="$y$ Position", value=gridDim[1]//2,min=0,max=gridDim[1]-1, layout=Layout(width='60%'))
z_slider = IntSlider(description="$z$ Position", value=gridDim[2]//2,min=0,max=gridDim[2]-1, layout=Layout(width='60%'))

# AFM plotting (interactive)
@interact(xIdx=x_slider, yIdx=y_slider, zIdx=z_slider)
def plot_AFM(xIdx,yIdx,zIdx):
    ### AFM plot
    figAFM, axAFM = plt.subplots(figsize=(6,6))
    axAFM.imshow(df[zIdx],origin='lower',cmap="gray", extent=extent)
    axAFM.axvline(xIdx*extent[1]/gridDim[0],c="r")
    axAFM.axhline(yIdx*extent[3]/gridDim[1],c="b")

# Height plotting (manual)
def plot():
    xIdx = x_slider.value
    yIdx = y_slider.value
    zIdx = z_slider.value

    # Limit for plots
    pos_lim = [np.min(pos[2][:,:,zIdx]),np.max(pos[2][:,:,zIdx])]
    disp_lim = [np.min(disp[2][:,:,zIdx]),np.max(disp[2][:,:,zIdx])]
    ### Height plots
    fig, ax = plt.subplots(3,2,figsize=(18,9))
    ## x-Axis
    # C-atom
    ax[0,0].plot(x_ref,pos[2][:,yIdx,zIdx], label="C, fixed $x$", c="b")
    ax[0,0].scatter(pos[0][:,yIdx,zIdx],pos[2][:,yIdx,zIdx], label="C, displaced $x$",
                    c=pos[1][:,yIdx,zIdx], s=20.0)
    ax[0,0].legend(loc="best")
    ax[0,0].set_ylim(pos_lim)
    ax[0,0].set_xlim(disx_lim)
    # O-atom
    ax[1,0].plot(x_ref,disp[2][:,yIdx,zIdx], label="O, fixed $x$", c="b")
    ax[1,0].scatter(disp[0][:,yIdx,zIdx],disp[2][:,yIdx,zIdx], label="O, displaced $x$",
                  c=disp[1][:,yIdx,zIdx], s=20.0)
    ax[1,0].legend(loc="best")
    ax[1,0].set_ylim(disp_lim)
    ax[1,0].set_xlim(disx_lim)
    # Atoms
    ax[2,0].scatter(atoms.get_positions()[:,0],atoms.get_positions()[:,2], s=50, c=colors)
    ax[2,0].set_xlim(disx_lim)
    ax[2,0].set_xlabel("$x$-Axis")
    ## y-Axis
    # C-atom
    ax[0,1].plot(y_ref,pos[2][xIdx,:,zIdx], label="C, fixed $y$", c="r")
    ax[0,1].scatter(pos[1][xIdx,:,zIdx],pos[2][xIdx,:,zIdx], label="C, displaced $y$",
                    c=pos[0][xIdx,:,zIdx], s=20.0)
    ax[0,1].legend(loc="best")
    ax[0,1].set_ylim(pos_lim)
    ax[0,1].set_xlim(disy_lim)
    # O-atom
    ax[1,1].plot(y_ref,disp[2][xIdx,:,zIdx], label="O, fixed $y$", c="r")
    ax[1,1].scatter(disp[1][xIdx,:,zIdx],disp[2][xIdx,:,zIdx], label="O, displaced $y$",
                    c=disp[0][xIdx,:,zIdx], s=20.0)
    ax[1,1].legend(loc="best")
    ax[1,1].set_ylim(disp_lim)
    ax[1,1].set_xlim(disy_lim)
    # Atoms
    ax[2,1].scatter(atoms.get_positions()[:,1],atoms.get_positions()[:,2], s=50, c=colors)
    ax[2,1].set_xlim(disy_lim)
    ax[2,1].set_xlabel("$y$-Axis")
interactive_plot = interactive(plot, {'manual': True})
# Fixing size to avoid jumping output
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot